In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from os import listdir
import json
from shutil import copyfile

import tensorflow as tf
import PIL
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

In [2]:
import zipfile
with zipfile.ZipFile('animal_photos.zip', 'r') as zip_ref:
    zip_ref.extractall(r'''C:\Users\franc\UOH\appliedAI''')


In [6]:
data_dir =r'C:\Users\franc\UOH\appliedAI\animal_photos'

In [7]:
batch_size = 32
img_height=180
img_width = 180

In [12]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset= 'training',
    seed=123,
    image_size=(img_height,img_width),
    batch_size=32
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset= 'validation',
    seed=123,
    image_size=(img_height,img_width),
    batch_size=32
)
class_names = train_ds.class_names
print(class_names)

Found 2423 files belonging to 3 classes.
Using 1939 files for training.
Found 2423 files belonging to 3 classes.
Using 484 files for validation.
['bear', 'horse', 'zebra']


In [13]:
autotune = tf.data.AUTOTUNE

In [24]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=autotune)
val_ds = val_ds.cache().prefetch(buffer_size= autotune)

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x,y: (normalization_layer(x),y))
image_batch,label_batch = next(iter(normalized_ds))
first_image = image_batch[5]
print(np.min(first_image),np.max(first_image))

num_classes = 3

0.0 1.0


In [22]:
model= Sequential([
    layers.experimental.preprocessing.Rescaling(1./255,input_shape=(img_height,img_width)),
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(num_classes)
])

ValueError: Input 0 of layer "conv2d_2" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 180, 180)